# MongoDB Vector Search - Haystack Integration

This notebook is a companion to the [Haystack Get Started](https://www.mongodb.com/docs/atlas/ai-integrations/haystack/) page. Refer to the page for set-up instructions and detailed explanations.

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/ai-integrations/haystack.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
pip install --quiet --upgrade mongodb-atlas-haystack voyage-embedders-haystack pymongo

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "<openai-api-key>"
os.environ["VOYAGE_API_KEY"] = "<voyage-api-key>"
os.environ["MONGO_CONNECTION_STRING"]= "<connection-string>"

In [ ]:
from pymongo import MongoClient
client = MongoClient(os.environ.get("MONGO_CONNECTION_STRING"))

In [ ]:
# Create your database and collection
db_name = "haystack_db"
collection_name = "test"
database = client[db_name]
database.create_collection(collection_name)

# Define collection
collection = client[db_name][collection_name]

In [ ]:
from pymongo.operations import SearchIndexModel
import time

# Create your index model, then create the search index
search_index_model = SearchIndexModel(
   definition={
      "fields": [
         {
            "type": "vector",
            "path": "embedding",
            "numDimensions": 1024,
            "similarity": "cosine"
         }
      ]
   },
   name="vector_index",
   type="vectorSearch"
)
result = collection.create_search_index(model=search_index_model)

# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None
if predicate is None:
  predicate = lambda index: index.get("queryable") is True

while True:
  indices = list(collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):
    break
  time.sleep(5)
print(result + " is ready for querying.")

In [ ]:
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore

document_store = MongoDBAtlasDocumentStore(
   database_name="haystack_db",
   collection_name="test",
   vector_search_index="vector_index",
   full_text_search_index="search_index" # Declared but not used in this example
)

In [ ]:
from haystack import Pipeline, Document
from haystack.document_stores.types import DuplicatePolicy
from haystack.components.writers import DocumentWriter
from haystack_integrations.components.embedders.voyage_embedders import VoyageDocumentEmbedder, VoyageTextEmbedder

# Create some example documents
documents = [
   Document(content="My name is Jean and I live in Paris."),
   Document(content="My name is Mark and I live in Berlin."),
   Document(content="My name is Giorgio and I live in Rome."),
]

# Initializing a document embedder to convert text content into vectorized form.
doc_embedder = VoyageDocumentEmbedder()

# Setting up a document writer to handle the insertion of documents into the MongoDB collection.
doc_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP)

# Creating a pipeline for indexing documents. The pipeline includes embedding and writing documents.
indexing_pipe = Pipeline()
indexing_pipe.add_component(instance=doc_embedder, name="doc_embedder")
indexing_pipe.add_component(instance=doc_writer, name="doc_writer")

# Connecting the components of the pipeline for document flow.
indexing_pipe.connect("doc_embedder.documents", "doc_writer.documents")

# Running the pipeline with the list of documents to index them in MongoDB.
indexing_pipe.run({"doc_embedder": {"documents": documents}})

## Basic RAG

In [ ]:
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever

# Template for generating prompts for a movie recommendation engine.
prompt_template = """
    You are an assistant allowed to use the following context documents.\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \nQuery: {{query}}
    \nAnswer:
"""

# Setting up a retrieval-augmented generation (RAG) pipeline for generating responses.
rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", VoyageTextEmbedder())

# Adding a component for retrieving related documents from MongoDB based on the query embedding.
rag_pipeline.add_component(instance=MongoDBAtlasEmbeddingRetriever(document_store=document_store,top_k=15), name="retriever")

# Building prompts based on retrieved documents to be used for generating responses.
rag_pipeline.add_component("prompt_builder", PromptBuilder(template=prompt_template, required_variables=["query", "documents"]))

# Adding a language model generator to produce the final text output.
rag_pipeline.add_component("llm", OpenAIGenerator())

# Connecting the components of the RAG pipeline to ensure proper data flow.
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

# Run the pipeline
query = "Where does Mark live?"
result = rag_pipeline.run(
  {
      "text_embedder": {"text": query},
      "prompt_builder": {"query": query},
  })
print(result['llm']['replies'][0])